# BCR4BP in Earth-Moon rotating frame

In [64]:
using DifferentialEquations
using Plots
using LinearAlgebra
using AstrodynamicsBase

In [2]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
include("../src/SailorMoon.jl")   # relative path to main file of module

Main.SailorMoon

In [17]:
param3b = SailorMoon.dyanmics_parameters()
lps = SailorMoon.lagrange_points(param3b.mu2)

5×6 Matrix{Float64}:
  0.836915   0.0       0.0  0.0  0.0  0.0
  1.15568    0.0       0.0  0.0  0.0  0.0
 -1.00506    0.0       0.0  0.0  0.0  0.0
  0.487849   0.866025  0.0  0.0  0.0  0.0
  0.487849  -0.866025  0.0  0.0  0.0  0.0

In [398]:
lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")
res.flag

Halo guess Az_km: 1200.0


1

In [399]:
res.x0

6-element Vector{Float64}:
  1.1226879860196828
  0.0
 -5.228885624166955e-26
  0.0
  0.1650993133797659
  0.0

In [400]:
ys0

6-element Vector{Float64}:
  0.2672179149228554
  0.2793813746152613
  2.469616446417975e-26
 -0.7710131455619899
 -0.5060428468351018
  1.3395041586794771e-25

In [401]:
x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1)
eps = 1e-6
if ys0[1] > 0
    svf = vcat(res.x0 + eps*ys0, 1.0)
else
    svf = vcat(res.x0 - eps*ys0, 1.0)
end

Linear stability ν = 618.7618470919092


7-element Vector{Float64}:
  1.1226882532375977
  2.793813746152613e-7
 -5.228883154550508e-26
 -7.710131455619898e-7
  0.16509880733691906
  1.339504158679477e-31
  1.0

In [472]:
θf = 2.65
tof = 25
eta = 0.4

tof_fwd = tof*eta
tof_bck = tof*(1 - eta)

15.0

In [473]:
tof* param3b.tstar/86400

108.70959021684592

In [474]:
mdot = 0.0
tmax = 0.0
τ, γ, β = 0.0, 0.0, 0.0
params = [param3b.mu2, param3b.mus, 
    θf, param3b.as, param3b.oms, τ, γ, β, mdot, tmax]

tspan = [0, -tof_bck]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_bcr4bp_thrust!, svf, tspan, params)
sol_bck = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12);

In [475]:
# departure state
rp = 6578/param3b.lstar
ra = 4.5*386400/param3b.lstar
sma = (rp + ra)/2
ecc = 1 - rp/sma
raan = 0.5
sv0_kep = [
    sma, ecc, 0.0, raan, 0.0, 0.0
]
θ0 = θf - param3b.oms*(tof_fwd + tof_bck)

sv0_i = AstrodynamicsBase.kep2cart(sv0_kep, param3b.mu1)
sv0 = vcat(
    inertial2rotating(sv0_i, θ0, 1.0) + [-param3b.mu2,0,0,0,0,0], 
    1.0
)

params = [param3b.mu2, param3b.mus, 
    θ0, param3b.as, param3b.oms, τ, γ, β, mdot, tmax]
tspan = [0, tof_fwd]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_bcr4bp_thrust!, sv0, tspan, params)
sol_fwd = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12);

In [476]:
sma, ecc

(2.2682074176562126, 0.9924623777771922)

In [477]:
tspan[2]*param3b.tstar/86400

43.48383608673837

In [478]:
plotly()

Plots.PlotlyBackend()

In [479]:
pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
scatter!(pcart, lps[:,1], lps[:,2], marker=:diamond, color=:red, label="LPs")
# trajectory
plot!(Array(sol_fwd)[1,:], Array(sol_fwd)[2,:], color=:deeppink, linewidth=1.5, label="fwd")
plot!(Array(sol_bck)[1,:], Array(sol_bck)[2,:], color=:purple, linewidth=1.5, label="bck")

pcart

### Plot in Earth-Moon inertial frame

In [485]:
sol_fwd_i = zeros(7,length(sol_fwd.u))
for (idx,sv) in enumerate(sol_fwd.u)
    theta = θ0 + sol_fwd.t[idx]
    sol_fwd_i[1:6,idx] = rotating2inertial(sv[1:6], theta, 1.0)
    sol_fwd_i[7,idx] = sv[7]
end

sol_bck_i = zeros(7,length(sol_bck.u))
for (idx,sv) in enumerate(sol_bck.u)
    theta = θf + sol_bck.t[idx]
    sol_bck_i[1:6,idx] = rotating2inertial(sv[1:6], theta, 1.0)
    sol_bck_i[7,idx] = sv[7]
end

In [486]:
pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)

# trajectory
plot!(sol_fwd_i[1,:], sol_fwd_i[2,:], color=:deeppink, linewidth=1.5, label="fwd")
plot!(sol_bck_i[1,:], sol_bck_i[2,:], color=:purple, linewidth=1.5, label="bck")

pcart

### Plot in Sun-Earth rotating frame

In [333]:
function planetmoon2sunplanet(
    state::Vector, μ_moon::Real, μ_sun::Real, theta::Real, om::Real,
    scale_l::Real, scale_v::Real, scale_t::Real, final_shift::Bool=false
)
    # 1. shift to Planet-centered, planet-moon rotating frame
    state_1 = state + [μ_moon, 0, 0, 0, 0, 0]
    # 2. convert to PLanet-centered, inertial
    state_2 = rotating2inertial(state_1, theta, om)
    # 3. re-scale
    state_3 = vcat(
        state_2[1:3] * scale_l,
        state_2[4:6] * scale_v
    )
    # 4. convert to Planet-centered, Sun-planet rotating frame
    theta_sp = scale_t * theta
    state_4 = inertial2rotating(state_3, theta_sp, om)
    # 5. shift to Sun-planet barycenter centered, Sun-planet rotating frame
    if final_shift == true
        return state_4 + [1 - μ_sun, 0, 0, 0, 0, 0]
    else
        return state_4
    end
end


planetmoon2sunplanet (generic function with 2 methods)

In [334]:
SE_system = R3BP.get_cr3bp_param(10,399)
EM_system = R3BP.get_cr3bp_param(399,301)

Main.R3BP.CR3BP_param(0.012150584269940354, 384400.0, 375190.2619517228, 403503.2355022598, 66182.92311059945)

In [335]:
scale_l = EM_system.lstar/SE_system.lstar
scale_v = (EM_system.lstar/EM_system.tstar) / (SE_system.lstar/SE_system.tstar)
scale_t = EM_system.tstar/SE_system.tstar

0.07469828599591845

In [336]:
θ0, θf

(37.08239997794829, 2.85)

In [345]:
sol_fwd_se = zeros(7,length(sol_fwd.u))
for (idx,sv) in enumerate(sol_fwd.u)
    theta = θ0 + sol_fwd.t[idx]
    sol_fwd_se[1:6,idx] = planetmoon2sunplanet(
        sv[1:6], param3b.mu2, param3b.mus, theta, 1.0, #param3b.oms,
        scale_l, scale_v, scale_t
    )
    sol_fwd_se[7,idx] = sv[7]
end

sol_bck_se = zeros(7,length(sol_bck.u))
for (idx,sv) in enumerate(sol_bck.u)
    theta = θf + sol_bck.t[idx]
    sol_bck_se[1:6,idx] = planetmoon2sunplanet(
        sv[1:6], param3b.mu2, param3b.mus, theta, 1.0, #param3b.oms,
        scale_l, scale_v, scale_t
    )
    sol_bck_se[7,idx] = sv[7]
end

In [346]:
#sol_bck.t

In [347]:
sol_bck_se - Array(sol_bck)

7×2377 Matrix{Float64}:
 -1.09084     -1.09084      …  -2.60288    -2.57848    -2.56278
  0.00136653   0.00183131      -2.13749    -2.17723    -2.20214
  0.0604662    0.0604654        0.211238    0.211021    0.210883
 -0.0213465   -0.0213088       -1.71354    -1.75687    -1.78409
 -0.304259    -0.304225         2.82504     2.8063      2.79414
 -5.16142e-7   0.000836102  …   0.0148115   0.0149035   0.0149612
  0.0          0.0              0.0         0.0         0.0

In [348]:
pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)

# trajectory
#plot!(Array(sol_fwd)[1,:], Array(sol_fwd)[2,:], color=:deeppink, linewidth=1.5, label="fwd")
plot!(sol_fwd_se[1,:], sol_fwd_se[2,:], color=:deeppink, linewidth=1.5, label="fwd")
plot!(sol_bck_se[1,:], sol_bck_se[2,:], color=:purple, linewidth=1.5, label="bck")

pcart